<center><h1>PantryPalette: Your Ingredient-Inspired Recipe Guide</h1></center>

# Problem Statement

Every day, households struggle with unused ingredients in their kitchens, often leading to food waste due
to a lack of inspiration or knowledge on how to use them effectively. Many individuals resort to takeout
or repetitive meals, simply because they don’t know what to cook with the ingredients they already have.


Meanwhile, traditional recipe search methods require manual browsing through extensive databases,
leading to time-consuming and frustrating experiences. Users with dietary restrictions, allergies, or
cultural food preferences face an additional challenge in finding recipes that align with their needs.

PantryPalette is designed to solve this problem by offering a seamless, ingredient-based recipe discovery
experience that helps users maximize their groceries, reduce food waste, and discover new meal ideas
with ease.

## Purpose

The purpose of the MLOps project is to develop an end-to-end data science solution that is usable by someone with no technical knowledge. 

It must include the following components
- A process map built which documents the process
- Data ingested from an online source
- A data repository and model repository
- A predictive model built off the data
- The model predictions

  a. Put into a Streamlit application

  b. Via a Docker deployment

  c. Which is accessible to users
  
- A model monitoring dashboard built
- Documentation for the model process and risks with the production

The purpose is not to have a complex model. It is meant to have a simpler model that works! All of this is to be done in teams of 2, along with individual users. Both the modeling teams and users will take part in the presentations

# Implementation

## Data Collection and Ingestion

- Get RecipeNLG data (static data) + Spoonacular API (dynamic data)

### Static Data Collection (RecipeNLG Dataset)

- Get RecipeNLG data (static data)

### Dynamic Data Integration (Spoonacular API)

- Get Spoonacular API (dynamic data)

### Data Preprocessing

- Clean & standardize ingredients

## Feature Engineering

- TF-IDF + Cosine Similarity

## Model Development

- Train NLP-based model

## Model Deployment

- Docker containerization & MLFlow

## UI Development

- Streamlit

## Model Monitoring

- PowerBI

<center><h4>Enjoy Your Perfect Recipe! 🍽️</h4></center>
<center>Designed By: Madhurya Shankar & Sandhya Kilari</center>